In [ ]:
import pandas as pd

# Step 1: Load the CSV
df = pd.read_csv('test.csv')


,EmployeeID,EmployeeName,Age,Gender,EducationLevel,ExperienceYears,DateOfJoining,Location,Department,Role,...,Performance_Composite,Potential_Composite,Behavior_Composite,Skill_Composite,Risk_Composite,FinalScore,FinalScore_0_100,Performance_Category,Potential_Category,NineBox
0,E1002,Employee_2,32,Other,Bachelors,18.1,27-01-2007,Central,IT,System Analyst,...,0.463595,0.424893,0.642857,0.500000,0.324356,0.475552,47.56,Medium,Medium,Medium-Medium
1,E1006,Employee_6,42,Other,Diploma,16.8,27-12-2005,East,IT,System Analyst,...,0.503507,0.479586,0.500000,0.555081,0.415676,0.496571,49.66,Medium,Medium,Medium-Medium
2,E1007,Employee_7,46,Female,Diploma,21.3,14-08-2007,South,IT,System Analyst,...,0.416273,0.523123,0.482143,0.641026,0.515599,0.485650,48.56,Medium,Medium,Medium-Medium
3,E1037,Employee_37,45,Male,Bachelors,3.5,22-06-2018,West,IT,System Analyst,...,0.485821,0.551583,0.392857,0.062203,0.468499,0.448377,44.84,Medium,Medium,Medium-Medium
4,E1040,Employee_40,27,Male,Diploma,8.5,05-03-2006,Central,IT,System Analyst,...,0.521047,0.606141,0.875000,0.263058,0.488596,0.572247,57.22,Medium,Medium,Medium-Medium
5,E1081,Employee_81,47,Female,Bachelors,6.3,18-09-2009,East,IT,System Analyst,...,0.420909,0.316962,0.303571,0.577398,0.443728,0.388914,38.89,Medium,Low,Low-Medium
6,E1086,Employee_86,38,Other,Masters,4.9,17-02-2008,East,IT,System Analyst,...,0.502682,0.507080,0.339286,0.566477,0.342135,0.477844,47.78,Medium,Medium,Medium-Medium
7,E1106,Employee_106,30,Female,PhD,13.0,20-09-2005,North,IT,System Analyst,...,0.385083,0.537859,0.339286,0.353751,0.319749,0.417646,41.76,Low,Medium,Medium-Low
8,E1109,Employee_109,54,Female,Bachelors,12.8,15-04-2018,North,IT,System Analyst,...,0.455228,0.459576,0.589286,0.512821,0.531519,0.486215,48.62,Medium,Medium,Medium-Medium
9,E1110,Employee_110,32,Other,Masters,19.9,28-06-2012,South,IT,System Analyst,...,0.440707,0.586966,0.000000,0.299620,0.705681,0.417618,41.76,Medium,Medium,Medium-Medium


In [ ]:
import pandas as pd
import numpy as np
import random

# Load the original dataset
df = pd.read_csv('test.csv')

# Number of new rows to generate
num_new_rows = 50

# Initialize dictionary for new data
new_data = {}

# Generate unique IDs
last_id = int(df['EmployeeID'].str.extract('(\d+)')[0].max())
new_ids = [f"E{i}" for i in range(last_id + 1, last_id + 1 + num_new_rows)]
new_data['EmployeeID'] = new_ids

# Generate Employee Names
new_data['EmployeeName'] = [f"Employee_{i}" for i in range(last_id + 1, last_id + 1 + num_new_rows)]

# Helper function to generate data based on column type
def generate_column_data(column_name, source_df, n):
    if column_name in ['EmployeeID', 'EmployeeName']:
        return None # Handled separately
    
    col_data = source_df[column_name]
    if pd.api.types.is_numeric_dtype(col_data):
        # For numeric, sample from range or normal distribution
        mean = col_data.mean()
        std = col_data.std()
        if pd.isna(std): std = 0
        min_val = col_data.min()
        max_val = col_data.max()
        
        # Generate random values
        if col_data.dtype == 'int64':
             return np.random.randint(min_val, max_val + 1, n)
        else:
             return np.round(np.random.uniform(min_val, max_val, n), 2)
             
    elif pd.api.types.is_object_dtype(col_data):
        # For categorical, sample from value counts
        values = col_data.dropna().unique()
        # Use weights based on frequency
        probs = col_data.value_counts(normalize=True)
        # Realign values and probs
        choice_values = probs.index.tolist()
        choice_weights = probs.values.tolist()
        return random.choices(choice_values, weights=choice_weights, k=n)
    
    return None

# Generate data for all columns except ID and Name
for col in df.columns:
    if col not in ['EmployeeID', 'EmployeeName']:
        new_data[col] = generate_column_data(col, df, num_new_rows)

# Create DataFrame
new_df = pd.DataFrame(new_data)

# Enforce consistency for SkillGapScore
# SkillGapScore = RequiredSkillScore - CurrentSkillScore
# Ensure we have these columns
if 'RequiredSkillScore' in new_df.columns and 'CurrentSkillScore' in new_df.columns:
    new_df['SkillGapScore'] = new_df['RequiredSkillScore'] - new_df['CurrentSkillScore']

# Save to CSV
output_filename = 'generated_employee_data.csv'
new_df.to_csv(output_filename, index=False)

print(f"Generated {output_filename} with {len(new_df)} rows.")
print(new_df.head().to_markdown(index=False, numalign="left", stralign="left"))

<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Avadhoot\AppData\Local\Temp\ipykernel_12116\4017142034.py:15: SyntaxWarning: invalid escape sequence '\d'
  last_id = int(df['EmployeeID'].str.extract('(\d+)')[0].max())
C:\Users\Avadhoot\AppData\Local\Temp\ipykernel_12116\4017142034.py:15: SyntaxWarning: invalid escape sequence '\d'
  last_id = int(df['EmployeeID'].str.extract('(\d+)')[0].max())


Generated generated_employee_data.csv with 50 rows.


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.